In [1]:
from amplpy import AMPL, DataFrame
import pandas as pd
import numpy as np
import os

# Load the ampl model

In [30]:
ampl = AMPL()
modelDirectory = "models"
modelName = "two_markets.mod"
mod_path = os.path.join(modelDirectory, modelName)
dataDirectory = "data"
ampl.read(mod_path)

# Assign set data

In [31]:
# Method to assign set data taking set name and a list as arguments
def assign_set_data(name,data):
    df = DataFrame(name)
    df.setColumn(name,data)
    ampl.setData(df,name)

In [32]:
intervals = pd.read_csv(os.path.join(dataDirectory, "intervals.csv"),skipinitialspace=True)
dffr = pd.read_csv(os.path.join(dataDirectory, "dffr.csv"),skipinitialspace=True)
da = pd.read_csv(os.path.join(dataDirectory, "da.csv"),skipinitialspace=True)

In [33]:
assign_set_data("INTERVALS",intervals.values[:,0])
assign_set_data("DFFR_PRICE",dffr.DFFR_PRICE.values)
assign_set_data("DA_PRICE",da.DA_PRICE.unique())

# Assign parameter data

In [34]:
ampl.getParameter('Cost').set(0)
ampl.getParameter('Ramp').set(999999)
ampl.getParameter('Ramp_DFFR').set(9999999)
ampl.getParameter('P_MAX').set(1)

In [35]:
n_DFFR_PRICE = len(dffr.DFFR_PRICE.unique())

df = pd.DataFrame([1/n_DFFR_PRICE for i in range(n_DFFR_PRICE)],
             columns=["p_R"],
             index = dffr.DFFR_PRICE.values)
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

n_DA_PRICE = len(da.DA_PRICE.unique())
df = pd.DataFrame([1/n_DA_PRICE for i in range(n_DA_PRICE)],
             columns=["p_DA"],
             index = da.DA_PRICE.unique())
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)



In [36]:
df = dffr.set_index("DFFR_PRICE")
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)

df = da.set_index(["DA_PRICE","INTERVALS"])
ampl_df = DataFrame.fromPandas(df)
ampl.setData(ampl_df)


# Solve the model

In [37]:
# Set ampl options
cplexopt = 'mipdisplay = 2 mipgap = .005'
knitro_options = 'outlev = 1 mip_outlevel = 1'
solver = 'knitro'
settings = {'solver': solver, 'knitro_options':knitro_options,'cplex_options': cplexopt}
for key in settings:
    ampl.setOption(key, settings[key])

In [38]:
ampl.solve()

KNITRO 8.0.0: outlev = 1
mip_outlevel = 1

          Commercial License
             KNITRO 8.0.0
          Ziena Optimization

KNITRO changing mip_method from AUTO to 1.
KNITRO changing mip_rootalg from AUTO to 1.
KNITRO changing mip_lpalg from AUTO to 3.
KNITRO changing mip_branchrule from AUTO to 2.
KNITRO changing mip_selectrule from AUTO to 2.
KNITRO changing mip_rounding from AUTO to 3.
KNITRO changing mip_heuristic from AUTO to 1.
KNITRO changing mip_pseudoinit from AUTO to 1.

Problem Characteristics
-----------------------
Objective goal:  Maximize
Number of variables:                   421
    bounded below:                       0
    bounded above:                       0
    bounded below and above:           421
    fixed:                               0
    free:                                0
Number of binary variables:            100
Number of integer variables:             0
Number of constraints:                 496
    linear equalities:                 121
    no

In [39]:
# Example of getting solution
p = ampl.getVariable("q_R")
p.getValues().toPandas()

,q_R.val
0,1.0


In [40]:
p = ampl.getVariable("q_DA")
p.getValues().toPandas()

,q_DA.val
"(1.0, 1.0)",1.0
"(1.0, 2.0)",1.0
"(1.0, 3.0)",1.0
"(1.0, 4.0)",1.0
"(1.0, 5.0)",1.0
"(1.0, 6.0)",1.0
"(2.0, 1.0)",1.0
"(2.0, 2.0)",1.0
"(2.0, 3.0)",1.0
"(2.0, 4.0)",1.0


In [43]:
# Example of getting solution
p = ampl.getVariable("P_Act")
p.getValues().toPandas()

,P_Act.val
"(1.0, 1.0, 1.0)",5.551115e-17
"(1.0, 1.0, 2.0)",5.551115e-17
"(1.0, 1.0, 3.0)",0.000000e+00
"(1.0, 1.0, 4.0)",0.000000e+00
"(1.0, 1.0, 5.0)",0.000000e+00
"(1.0, 1.0, 6.0)",5.551115e-17
"(1.0, 2.0, 1.0)",0.000000e+00
"(1.0, 2.0, 2.0)",5.551115e-17
"(1.0, 2.0, 3.0)",0.000000e+00
"(1.0, 2.0, 4.0)",0.000000e+00
